In [48]:
import pandas as pd
import numpy as np
import glob
import requests
import json
import torch
import clip
import pymilvus
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    db
)

In [2]:
titles_data = pd.read_parquet('title_embeddings/metadata/metadata_0.parquet')
IMAGES_URL_DIRECTORY = 'image_folder'

In [3]:
titles_data.head()

,caption
0,"Howard LC0008 Leather Conditioner, 8-Ounce (4-..."
1,Yes to Tomatoes Detoxifying Charcoal Cleanser ...
2,Avon Big & False Lash Mascara - Brown/Black by...
3,Amybria New Silver Plated Full Crystal Inlay B...
4,Lucky 7 Shaving Starter Set - Seven Blade Shav...


In [4]:
len(titles_data)

112590

In [5]:
title_embeddings = np.load('title_embeddings/text_emb/text_emb_0.npy')

In [6]:
title_embeddings.shape

(112590, 512)

In [7]:
images_data = pd.read_parquet('image_folder/00000.parquet')

In [8]:
images_data.head()

,url,key,status,error_message,width,height,original_width,original_height,exif,sha256
0,https://m.media-amazon.com/images/I/517uoA+-gz...,000000016,success,None,224.0,224.0,1005.0,1005.0,{},d9bbcad7ec92ac4883378b3c4c02115fb368e85918fb56...
1,https://m.media-amazon.com/images/I/510BWq7O95...,000000015,success,None,224.0,224.0,1005.0,1005.0,{},39d9ae4617737dcbf3d8ba0c655e8b5ac70cfc955dee25...
2,https://m.media-amazon.com/images/I/71g1lP0pMb...,000000002,success,None,224.0,224.0,540.0,1500.0,{},fc7df6df54aeb9b681713a5850d74591f011476343e135...
3,https://m.media-amazon.com/images/I/513VyUt42Y...,000000018,success,None,224.0,224.0,1005.0,1005.0,{},3800225ab55c760e8ec749f907c87382afd02d8c85489f...
4,https://m.media-amazon.com/images/I/511v5MygzT...,000000017,success,None,224.0,224.0,1005.0,1005.0,{},f68a9d335bbc13c5f22796d0421a69dd03bc3949ee5dff...


In [9]:
images_data.shape

(10000, 10)

In [10]:
url_key_dict = images_data.set_index('url')['key'].to_dict()

# Display the dictionary
print(url_key_dict)

{'https://m.media-amazon.com/images/I/517uoA+-gzL._SL1005_.jpg': '000000016', 'https://m.media-amazon.com/images/I/510BWq7O95L._SL1005_.jpg': '000000015', 'https://m.media-amazon.com/images/I/71g1lP0pMbL._SL1500_.jpg': '000000002', 'https://m.media-amazon.com/images/I/513VyUt42YL._SL1005_.jpg': '000000018', 'https://m.media-amazon.com/images/I/511v5MygzTL._SL1005_.jpg': '000000017', 'https://m.media-amazon.com/images/I/31bz+uqzWCL.jpg': '000000005', 'https://m.media-amazon.com/images/I/71i77AuI9xL._SL1500_.jpg': '000000001', 'https://m.media-amazon.com/images/I/61NS1lONhzL._SL1001_.jpg': '000000007', 'https://m.media-amazon.com/images/I/51+ERW-6D0L._SL1005_.jpg': '000000021', 'https://m.media-amazon.com/images/I/81OqvR94isL._SL1500_.jpg': '000000003', 'https://m.media-amazon.com/images/I/51f2sDEVH7L._SL1005_.jpg': '000000020', 'https://m.media-amazon.com/images/I/61m-71vCbCL._SL1001_.jpg': '000000012', 'https://m.media-amazon.com/images/I/31TgqAZ8kQL.jpg': '000000013', 'https://m.media

In [19]:
image_embeddings_meta_data = pd.read_parquet('image_embeddings_complete\metadata\metadata_0.parquet')
image_embeddings_meta_data.head(520000)


,image_path
0,image_folder/00000/000000000.jpg
1,image_folder/00000/000000001.jpg
2,image_folder/00000/000000002.jpg
3,image_folder/00000/000000003.jpg
4,image_folder/00000/000000004.jpg
...,...
519995,image_folder/00052/000521073.jpg
519996,image_folder/00052/000521074.jpg
519997,image_folder/00052/000521075.jpg
519998,image_folder/00052/000521076.jpg


In [24]:
image_embeddings_meta_data['key'] = image_embeddings_meta_data['image_path'].str.extract(r'(\d{9})')
image_embeddings_meta_data.head(520000)


,image_path,key
0,image_folder/00000/000000000.jpg,000000000
1,image_folder/00000/000000001.jpg,000000001
2,image_folder/00000/000000002.jpg,000000002
3,image_folder/00000/000000003.jpg,000000003
4,image_folder/00000/000000004.jpg,000000004
...,...,...
519995,image_folder/00052/000521073.jpg,000521073
519996,image_folder/00052/000521074.jpg,000521074
519997,image_folder/00052/000521075.jpg,000521075
519998,image_folder/00052/000521076.jpg,000521076


In [26]:
image_embeddings_meta_data.shape

(548083, 1)

In [46]:
index_value = image_embeddings_meta_data.loc[image_embeddings_meta_data['key'] == '000010004'].index
index_value.empty

True

In [33]:
d = {'dfadf':21}
d.update({'ds':32})
d

{'dfadf': 21, 'ds': 32}

In [20]:
def creating_images_url_dictionary(path = IMAGES_URL_DIRECTORY):
# Initialize an empty dictionary
    url_key_dict = {}

    # List of file paths (adjust the path and extension as needed)
    file_paths = glob.glob(f'{path}/*.parquet')
    
    # print(len(file_paths))
    
    for file_path in file_paths:
        # Read the file into a DataFrame
        data = pd.read_parquet(file_path)
        
        # Ensure the key column is in string format
        data['key'] = data['key'].astype(str)
        
        # Update the dictionary with data from the current file
        url_key_dict.update(data.set_index('url')['key'].to_dict())
    
    return url_key_dict

In [21]:
images_url_dictionary = creating_images_url_dictionary()

55


In [22]:
print(len(images_url_dictionary))

478384


In [18]:
print(images_url_dictionary['https://m.media-amazon.com/images/I/71jV-W1EJHS._SL1001_.jpg'])

000276268


In [44]:
image_embeddings = np.load('image_embeddings_complete\img_emb\img_emb_0.npy')
image_embeddings.shape

(548083, 512)